In [1]:
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as Dataset
import torchvision.transforms.functional as TVF
import torch.autograd as autograd
from torch.utils.data import Dataset, DataLoader

from torchvision.models import vgg16 
from PIL import Image

import nltk
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

import psutil
import time

In [3]:
t1 = torch.zeros(3,4)
t1.cuda()

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], device='cuda:0')

In [4]:
psutil.virtual_memory()

svmem(total=17046654976, available=10860032000, percent=36.3, used=6186622976, free=10860032000)

In [5]:
columns = ['filename', 'caption']
caption_data = pd.read_csv('./Flickr8k_text/Flickr8k_token.txt' , 
                           sep = '\t' ,
                          header = None,
                          names = columns)
caption_data.head()

,filename,caption
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...


In [6]:
caption_data['caption_num'] = caption_data['filename'].str.split('#',expand=True)[1]
caption_data.head()

,filename,caption,caption_num
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...,0
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .,1
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .,2
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...,3
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...,4


In [7]:
caption_data.shape

(40460, 3)

In [8]:
40460 / 5

8092.0

In [9]:
caption_data['word_count'] = caption_data['caption'].apply(lambda x: len(str(x).split(" ")))
caption_data['word_count'].describe(percentiles = [0.25 , 0.5 , 0.75 , 0.9 , 0.95 , 0.99])

count    40460.000000
mean        11.782155
std          3.885175
min          1.000000
25%          9.000000
50%         11.000000
75%         14.000000
90%         17.000000
95%         19.000000
99%         23.000000
max         38.000000
Name: word_count, dtype: float64

In [10]:
word_idx = 0
word_to_idx_dict = {}
idx_to_word_dict = {}
sentence_list = []
for i in caption_data['caption']:
    text = i.lower().split(' ')
    sentence_list.append(text)
    for j in text:
        if word_to_idx_dict.get(j,0) == 0:
            word_to_idx_dict[j] = word_idx
            idx_to_word_dict[word_idx] = j
            word_idx+=1
        else: 
            pass

In [11]:
print('word_to_idx_dict : {}'.format(len(word_to_idx_dict)))
print('idx_to_word_dict : {}'.format(len(idx_to_word_dict)))

word_to_idx_dict : 8918
idx_to_word_dict : 8919


In [12]:
idx_to_word_dict[8918]

'patterns'

In [13]:
word_to_idx_dict['patterns']

8918

In [14]:
#vgg16()

In [15]:
#model_vgg16 = vgg16(pretrained=True)
#model_vgg16

In [16]:
#model_vgg16.classifier = nn.Linear(in_features=25088, out_features=4096, bias=True)

In [17]:
p = torchvision.transforms.Compose([torchvision.transforms.Resize((100,75))])
image_dict = {}
ctr = 0
path =  'D:\\Data_Science\\image_captioning\\Flickr8k_Dataset\\Flicker8k_Dataset'
total_files = len(os.listdir(path))
st_time = time.time()
for file in os.listdir(path):
    image = Image.open(path + '\\' + file)
    image = p(image)
    image_dict[file] = TVF.to_tensor(image)
    ctr+=1
    if ctr % 100 == 0:
        print(' file : {} / {} in {} seconds'.format(ctr , total_files , (time.time() - st_time)))
        st_time = time.time()

 file : 100 / 8091 in 0.36804676055908203 seconds
 file : 200 / 8091 in 0.3048670291900635 seconds
 file : 300 / 8091 in 0.3027181625366211 seconds
 file : 400 / 8091 in 0.3563570976257324 seconds
 file : 500 / 8091 in 0.2905843257904053 seconds
 file : 600 / 8091 in 0.3166837692260742 seconds
 file : 700 / 8091 in 0.31396484375 seconds
 file : 800 / 8091 in 0.32195329666137695 seconds
 file : 900 / 8091 in 0.29948925971984863 seconds
 file : 1000 / 8091 in 0.3249082565307617 seconds
 file : 1100 / 8091 in 0.31793689727783203 seconds
 file : 1200 / 8091 in 0.3161802291870117 seconds
 file : 1300 / 8091 in 0.3889596462249756 seconds
 file : 1400 / 8091 in 0.3112001419067383 seconds
 file : 1500 / 8091 in 0.3111598491668701 seconds
 file : 1600 / 8091 in 0.3111913204193115 seconds
 file : 1700 / 8091 in 0.30718326568603516 seconds
 file : 1800 / 8091 in 0.3022148609161377 seconds
 file : 1900 / 8091 in 0.3091914653778076 seconds
 file : 2000 / 8091 in 0.32016754150390625 seconds
 file : 

In [18]:
seq_length = 38
vocab_size = len(word_to_idx_dict)
embed_size = 4096
hidden_size = embed_size

In [19]:
class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        
        # get the pretrained vgg16 model
        self.vgg16 = vgg16(pretrained=True)
        
        # replace the classifier with a fully connected layer
        self.vgg16.classifier = nn.Linear(in_features=25088, out_features=embed_size, bias=True)
        
        # add another fully connected layer
        self.fc = nn.Linear(in_features=embed_size, out_features=embed_size)
        
        # dropout layer
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        
        # get the embeddings from vgg16
        x = self.vgg16(x)
        x = F.relu(x)
        
        # pass through dropout
        x = self.dropout(x)
        
        # pass through the fully connected
        x = self.fc(x)
        
        return x

In [29]:
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1):
        super(DecoderRNN, self).__init__()
        
        # define the properties
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        
        # lstm cell
        self.lstm_cell = nn.LSTMCell(input_size=embed_size, hidden_size=hidden_size)
    
        # output fully connected layer
        self.fc_out = nn.Linear(in_features=self.hidden_size, out_features=self.vocab_size)
    
    def forward(self, embedding, caption_embed):
        
        outputs = torch.empty((batch_size , caption_embed.size(1) , vocab_size))
        hidden_state = torch.zeros((batch_size, hidden_size)).to(device)
        cell_state = torch.zeros((batch_size, hidden_size)).to(device)
        
        # pass the caption word by word
        for t in range(caption_embed.size(1)):

            # for the first time step the input is the embedding vector
            if t == 0:
                hidden_state , cell_state = self.lstm_cell(embedding , (hidden_state, cell_state))
                
            #fot t>0 , input is previous hidden state and cell states
            else :
                hidden_state, cell_state = self.lstm_cell(hidden_state, (hidden_state, cell_state))
            
            #Pass through output layer to match vocab size
            out = self.fc_out(hidden_state)
            
            # build the output tensor
            outputs[:, t, :] = out
    
        return outputs

In [30]:
encoder_rnn = EncoderCNN(embed_size = embed_size)
decoder_rnn = DecoderRNN(embed_size = embed_size , hidden_size = embed_size , vocab_size = vocab_size)

In [33]:
if torch.cuda.is_available():
    print('GPU available')
    device = torch.device("cuda")
else :
    device = torch.device("cpu")
    
device = torch.device("cpu")

GPU available


In [34]:
epochs = 1
batch = 0
print_every = 1
batch_size = 1

learning_rate = 0.001
optimizer = torch.optim.Adam(decoder_rnn.parameters(), lr = learning_rate)

encoder_rnn.to(device)
decoder_rnn.to(device)

train_losses = []
val_losses = []

for epoch in range(epochs):
    
    for df_idx in caption_data.head().index:
        file_name = caption_data.iloc[df_idx].filename.split('#')[0]
        print(file_name)
        
        image = image_dict[file_name]
        image = image.unsqueeze(0).to(device)
        
        optimizer.zero_grad()
        embedding = encoder_rnn(image)
    
        caption = caption_data.iloc[0].caption
        caption = caption.lower().split(' ')
        
        # init the hidden and cell states to zeros
        caption_embed = torch.zeros((batch_size ,len(caption) , vocab_size) ).to(device)
        
        idx = 0
        batch = 0
        for words in caption:
            word_idx = word_to_idx_dict[words]
            caption_embed[batch][idx][word_idx] = 1
            idx+=1
        
        y_pred = decoder_rnn(embedding = embedding , caption_embed = caption_embed)
        loss = 
    
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
        if batch % print_every == 0:
            print('epoch : {} \t batch number : {} \t train loss : {}'.format(epoch, batch,loss.item()))
            
        batch +=1
        
        break
    break

1000268201_693b08cb0e.jpg


NameError: name 'loss' is not defined

In [91]:
embedding = encoder_rnn(x)

In [92]:
embedding.shape

torch.Size([1, 4096])

In [94]:
caption = caption_data.iloc[0].caption
caption = caption.lower().split(' ')
        
# init the hidden and cell states to zeros
caption_embed = torch.zeros((batch_size ,len(caption) , vocab_size) ).to(device)
        
idx = 0
batch = 0
for words in caption:
    word_idx = word_to_idx_dict[words]
    caption_embed[batch][idx][word_idx] = 1
    idx+=1

In [95]:
y_pred = decoder_rnn(embedding = embedding , caption_embed = caption_embed) 

In [96]:
y_pred.shape

torch.Size([1, 18, 8918])

In [19]:
decoder_input_size = embedding.shape[1]
decoder_input_size

4096

In [103]:
y_pred.shape

torch.Size([1, 18, 8918])

In [104]:
caption_embed.shape

torch.Size([1, 18, 8918])

In [115]:
loss = loss_fn(y_pred , caption_embed)

In [116]:
loss

tensor(0.0002, grad_fn=<MseLossBackward>)

In [98]:
lstm_cell = nn.LSTMCell(input_size=decoder_input_size, hidden_size=decoder_input_size)
fc_out = nn.Linear(in_features=decoder_input_size, out_features=vocab_size)

In [21]:
batch_size = 1
hidden_state = torch.zeros((batch_size , decoder_input_size))
cell_state = torch.zeros((batch_size , decoder_input_size))

In [22]:
hidden_state , cell_state = lstm_cell(embedding , ((hidden_state, cell_state)))

In [23]:
out = fc_out(hidden_state)

In [24]:
out.shape

torch.Size([1, 8918])

In [25]:
idx_to_word_dict[torch.argmax(out.softmax(dim = 1)).item()]

'pinkish'

In [26]:
actual_caption = caption_data.iloc[0].caption
predicted_sentence = []
actual_caption = actual_caption.lower().split(' ')
actual_caption

['a',
 'child',
 'in',
 'a',
 'pink',
 'dress',
 'is',
 'climbing',
 'up',
 'a',
 'set',
 'of',
 'stairs',
 'in',
 'an',
 'entry',
 'way',
 '.']

In [27]:
for t in range(len(actual_caption)):
    if t == 0:
        hidden_state , cell_state = lstm_cell(embedding , ((hidden_state, cell_state)))
    else :
        hidden_state, cell_state = lstm_cell(hidden_state, (hidden_state, cell_state))
        
    out = fc_out(hidden_state)
    word = idx_to_word_dict[torch.argmax(out.softmax(dim = 1)).item()]
    predicted_sentence.append(word)
    print('t :  {}\tword : {}'.format(t,word))
        
#predicted_sentence 

t :  0	word : expressions
t :  1	word : expressions
t :  2	word : baptized
t :  3	word : opposition
t :  4	word : opposition
t :  5	word : bras
t :  6	word : bras
t :  7	word : bras
t :  8	word : bras
t :  9	word : ceremonial
t :  10	word : ceremonial
t :  11	word : ceremonial
t :  12	word : ceremonial
t :  13	word : ceremonial
t :  14	word : ceremonial
t :  15	word : ceremonial
t :  16	word : ceremonial
t :  17	word : ceremonial


In [28]:
zero_tensor = torch.zeros(8918)
word_idx = word_to_idx_dict['a']
word_tensor = zero_tensor
word_tensor[word_idx] = 1

In [31]:
sentence_embed = torch.zeros((batch_size ,len(actual_caption) , vocab_size) )
sentence_embed.shape

torch.Size([1, 18, 8918])

In [32]:
idx = 0
batch = 0
for words in actual_caption:
    word_idx = word_to_idx_dict[words]
    sentence_embed[batch][idx][word_idx] = 1
    idx+=1

In [33]:
sentence_embed.shape[1]

torch.Size([1, 18, 8918])

18

In [81]:
train_set = torchvision.datasets.Flickr8k(root = 'D:/Data_Science/image_captioning/Flickr8k_Dataset/Flickr8k_Dataset',
                                             ann_file = 'D:/Data_Science/image_captioning/Flickr8k_text/Flickr8k_token.txt',
                                             transform = torchvision.transforms.ToTensor)

In [82]:
train_set_loader = torch.utils.data.DataLoader(train_set,batch_size=4,shuffle=False,num_workers=2)

In [83]:
var = iter(next(train_set_loader))

TypeError: 'DataLoader' object is not an iterator

In [78]:
next(iter(train_set_loader))

StopIteration: 

 file : 100 / 8091 in 0.30092740058898926 seconds
 file : 200 / 8091 in 0.31882214546203613 seconds
 file : 300 / 8091 in 0.2999558448791504 seconds
 file : 400 / 8091 in 0.33214545249938965 seconds
 file : 500 / 8091 in 0.2991297245025635 seconds
 file : 600 / 8091 in 0.31565427780151367 seconds
 file : 700 / 8091 in 0.3010292053222656 seconds
 file : 800 / 8091 in 0.31662845611572266 seconds
 file : 900 / 8091 in 0.32251906394958496 seconds
 file : 1000 / 8091 in 0.31117987632751465 seconds
 file : 1100 / 8091 in 0.31917476654052734 seconds
 file : 1200 / 8091 in 0.2930927276611328 seconds
 file : 1300 / 8091 in 0.2999758720397949 seconds
 file : 1400 / 8091 in 0.3000307083129883 seconds
 file : 1500 / 8091 in 0.301058292388916 seconds
 file : 1600 / 8091 in 0.2999439239501953 seconds
 file : 1700 / 8091 in 0.3009648323059082 seconds
 file : 1800 / 8091 in 0.298907995223999 seconds
 file : 1900 / 8091 in 0.31517505645751953 seconds
 file : 2000 / 8091 in 0.32416534423828125 seconds
 

In [41]:
#image_dict

In [9]:
image.size

(375, 500)

In [21]:
375 / 500

0.75

In [22]:
375 / 5

75.0

In [39]:
caption_data.filename[0].split()

'1000268201_693b08cb0e.jpg#0'